In [1]:
import pandas as pd
import numpy as np
import pickle
ratings = pd.read_csv('C:/Users/prtyagi/Desktop/New folder/archive1/ratings_small.csv')
ratings.drop('timestamp',axis='columns', inplace=True)
movies = pd.read_csv("C:/Users/prtyagi/Desktop/New folder/archive1/movies_metadata.csv")
movies = movies.drop([19730, 29503, 35587])
movies = movies[['id', 'title']]
movies['id'] = movies['id'].astype('int')
dataset = pd.merge(ratings, movies, left_on='movieId', right_on='id')
dataset = dataset[['userId','title', 'rating']]
dataset.columns = ['userID', 'itemID', 'rating']
dataset['reviews'] = dataset.groupby(['itemID'])['rating'].transform('count')
dataset= dataset[dataset.reviews>25][['userID', 'itemID', 'rating']]
with open('popularity_data.pickle', 'rb') as handle:
    popularity_data = pickle.load(handle)
dataset = pd.merge(dataset, popularity_data, left_on='itemID', right_on='title')
import pickle

with open("dataset.pickle", 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
dataset = dataset[['userID','itemID', 'rating']]
dataset.head()

C:\Users\prtyagi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,userID,itemID,rating
0,1,Rocky III,2.5
1,4,Rocky III,4.0
2,7,Rocky III,3.0
3,19,Rocky III,4.0
4,21,Rocky III,3.0


In [2]:
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(dataset, reader)

In [3]:
unique_ids = dataset['itemID'].unique()
# get the list of the ids that the userid 1001 has rated
iids650 = dataset.loc[dataset['userID']==650, 'itemID']
# remove the rated movies for the recommendations
movies_to_predict = np.setdiff1d(unique_ids,iids650)

In [4]:
algo = SVDpp()
algo.fit(data.build_full_trainset())
my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=650,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
200,The Good Shepherd,4.424342
52,Dead Man,4.414310
241,While You Were Sleeping,4.410967
73,Galaxy Quest,4.387407
164,Sleepless in Seattle,4.375649
195,The Dreamers,4.314523
60,"Don't Worry, I'm Fine",4.292771
112,Men in Black II,4.288496
80,Hard Target,4.281493
37,Bonnie and Clyde,4.276944


In [5]:
import pickle

with open("svdpp.pickle", 'wb') as handle:
    pickle.dump(algo, handle, protocol=pickle.HIGHEST_PROTOCOL)